In [4]:
# setup dataset + index
from datasets import Dataset
import numpy as np
import faiss
retrieval_vector_size = 768

dataset = Dataset.from_dict(
    {
        "id": [str(i) for i in range(10)],
        "text": ["My favourite number is 3455", "The secret word is FROG"] * 5,
        "embeddings": [
            0.1 * np.ones(retrieval_vector_size),
            0.9 * np.ones(retrieval_vector_size),
        ] * 5,
    }
)

dataset.add_faiss_index("embeddings", metric_type=faiss.METRIC_INNER_PRODUCT)
from src.transformers.models.atlas.retrieval_atlas import AtlasRetrieverIndex, AtlasConfig, AtlasTokenizer

config = AtlasConfig.from_pretrained("./data/atlas-pretrained")
tokenizer = AtlasTokenizer.from_pretrained("./data/atlas-pretrained", config=config)

retriever_index = AtlasRetrieverIndex(config, tokenizer, dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# for some reason, src.transformers.models.atlas.modeling_atlas as import causes an error here
# model doesn't load right?
from transformers import AtlasModel

atlas = AtlasModel.from_pretrained('data/atlas-pretrained', retriever_index=retriever_index)

In [6]:


tokens = tokenizer.generator("Hello world query", return_tensors="pt")
labels = tokenizer.generator("Hello world answer", return_tensors="pt")
query_tokens = tokenizer.retriever("Hello world query", return_tensors="pt")

labels[labels == tokenizer.generator.pad_token_id] = -100


atlas.forward(
    input_ids=tokens.input_ids,
    attention_mask=tokens.attention_mask,
    labels=labels.input_ids,
    query_input_ids=query_tokens.input_ids,
    query_attention_mask=query_tokens.attention_mask,
    top_k=2,
)


(None, tensor(10.7796, grad_fn=<NllLossBackward0>))